In [41]:
import pandas as pd
from joblib import dump

from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score, recall_score, precision_score, accuracy_score

from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier

In [42]:
dfo = pd.read_csv("..\\Data\\SmA-Four-Tank-Batch-Process_V2.csv",sep=';')
dfo['timestamp'] = pd.to_datetime(dfo['timestamp'])

Pre-processing und Feature "Dev"

In [43]:
df = dfo.copy()

df = df.drop(columns = ["timestamp","CuStepNo ValueY"])
c = list(df.columns)
c[0] = "Dev"
df.columns = c
df=df[df['Dev'] != 0]
df['Dev'] = df['Dev']!=1
df['Dev'] = df['Dev'].astype('int')
df_clean = df.copy()

In [44]:
df_clean.to_csv('..\\Data\\data_01_clean.csv')
df_clean = df_clean.sample(250000,random_state=42)
X_train,X_test,y_train,y_test = train_test_split(df_clean.drop(columns = ['Dev']), df_clean['Dev'], shuffle = False, random_state = 42)

In [45]:
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train,y_train)
dump(model, "rf.pkl")

['rf.pkl']

# Metriken der Trainings-Daten

In [46]:
y_pred_train = model.predict(X_train)
print(f'R2: {r2_score(y_train,y_pred_train)}\nAccuracy: {accuracy_score(y_train,y_pred_train)}\nRecall: {recall_score(y_train,y_pred_train)}\nPrecision: {precision_score(y_train,y_pred_train)}')

R2: 1.0
Accuracy: 1.0
Recall: 1.0
Precision: 1.0


In [47]:

tn, fp, fn, tp = confusion_matrix(y_train,y_pred_train).ravel()
print(f'TN: {tn}')
print(f'FP: {fp}')
print(f'FN: {fn}')
print(f'TP: {tp}')

TN: 25075
FP: 0
FN: 0
TP: 162425


# Metriken der Test-Daten

In [48]:
y_pred = model.predict(X_test)
print(f'R2: {r2_score(y_test,y_pred)}\nAccuracy: {accuracy_score(y_test,y_pred)}\nRecall: {recall_score(y_test,y_pred)}\nPrecision: {precision_score(y_test,y_pred)}')

R2: 0.8430014475194227
Accuracy: 0.981936
Recall: 0.9914405357044034
Precision: 0.987777981988605


In [49]:
y_pred=model.predict(X_test)
tn, fp, fn, tp = confusion_matrix(y_test,y_pred).ravel()
print(f'TN: {tn}')
print(f'FP: {fp}')
print(f'FN: {fn}')
print(f'TP: {tp}')

TN: 7626
FP: 665
FN: 464
TP: 53745


Hinweis: <br> 
Durch die Ergänzung der Vorhersage durch eine zeitliche Klassifizierung (dargestellt in "zeitliche_Klassifizierung.ipynb"), können dann alle fehlerhaften Batches als fehlerhaft klassifiziert werden. <br> 
Dies müsste in einem produktiven System mit Livedaten in einer Software umgesetzt werden. <br>